In [1]:
import tensorflow as tf

In [2]:
tf.enable_eager_execution()

## Layer Examples

In [3]:
layer1 = tf.keras.layers.Dense(100)
layer2 = tf.keras.layers.Dense(10, input_shape=(None, 5))

layer1, layer2

(<tensorflow.python.keras.layers.core.Dense at 0x7f1d05713be0>,
 <tensorflow.python.keras.layers.core.Dense at 0x7f1d05713c18>)

In [4]:
data = tf.zeros([10, 5])
data

<tf.Tensor: id=3, shape=(10, 5), dtype=float32, numpy=
array([[0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.]], dtype=float32)>

In [5]:
layer2(data)

<tf.Tensor: id=31, shape=(10, 10), dtype=float32, numpy=
array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]], dtype=float32)>

## Custom Layers

In [6]:
class MyDenseLayer(tf.keras.layers.Layer):
    
    def __init__(self, num_outputs):
        super(MyDenseLayer, self).__init__()
        self.num_outputs = num_outputs
    
    def build(self, input_shape):
        self.kernel = self.add_variable("kernel", shape=[int(input_shape[-1]), self.num_outputs])
        
    def call(self, input):
        return tf.matmul(input, self.kernel)
    
MyDenseLayer

__main__.MyDenseLayer

In [7]:
layer = MyDenseLayer(10)
layer

In [8]:
layer(data)

<tf.Tensor: id=46, shape=(10, 10), dtype=float32, numpy=
array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]], dtype=float32)>

In [9]:
layer.variables

[<tf.Variable 'my_dense_layer/kernel:0' shape=(5, 10) dtype=float32, numpy=
 array([[-0.5958435 , -0.3979122 , -0.32803258, -0.20822579, -0.50205773,
          0.3408839 ,  0.29653877, -0.13965473,  0.3553046 ,  0.5872629 ],
        [-0.20231879, -0.31852323, -0.230811  , -0.34674478, -0.38086113,
          0.50338787,  0.04151243,  0.1344009 , -0.14484504, -0.21437061],
        [ 0.10367221, -0.43133086, -0.3419631 ,  0.5300018 ,  0.05390835,
          0.31575793,  0.51888305,  0.35846323, -0.0758943 , -0.3993916 ],
        [-0.6222625 , -0.3671862 ,  0.01019257, -0.30387166,  0.60836774,
         -0.4544518 , -0.01821411, -0.3993284 , -0.2116653 ,  0.03606755],
        [ 0.59466726, -0.40445495,  0.51253575, -0.57618827, -0.32920587,
         -0.58777034,  0.04794163, -0.02760315, -0.47527292, -0.4621351 ]],
       dtype=float32)>]

## Composing Layers

In [10]:
class ResnetIdentityBlock(tf.keras.Model):
    
    def __init__(self, kernel_size, filters):
        
        super(ResnetIdentityBlock, self).__init__(name="")
        
        filters1, filters2, filters3 = filters
        
        self.conv2a = tf.keras.layers.Conv2D(filters1, (1, 1))
        self.bn2a = tf.keras.layers.BatchNormalization()
        self.conv2b = tf.keras.layers.Conv2D(filters2, kernel_size, padding='same')
        self.bn2b = tf.keras.layers.BatchNormalization()
        self.conv2c = tf.keras.layers.Conv2D(filters3, (1, 1))
        self.bn2c = tf.keras.layers.BatchNormalization()
        
    def call(self, input_tensor, training=False):
        x = self.conv2a(input_tensor)
        x = self.bn2a(x, training=training)
        x = tf.nn.relu(x)
        x = self.conv2b(x)
        x = self.bn2b(x, training=training)
        x = tf.nn.relu(x)
        x = self.conv2c(x)
        x = self.bn2c(x, training=training)
        x = tf.nn.relu(x)
        return x
        
ResnetIdentityBlock

__main__.ResnetIdentityBlock

In [11]:
block = ResnetIdentityBlock(1, [1, 2, 3])
block

In [13]:
data = tf.zeros([1, 2, 3, 3])
data

<tf.Tensor: id=53, shape=(1, 2, 3, 3), dtype=float32, numpy=
array([[[[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]],

        [[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]]]], dtype=float32)>

In [14]:
block(data)

<tf.Tensor: id=281, shape=(1, 2, 3, 3), dtype=float32, numpy=
array([[[[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]],

        [[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]]]], dtype=float32)>